**Imports**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

**Load CIFAR-100 data using PyTorch**

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 169001437/169001437 [00:05<00:00, 30024393.04it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


**Define the CNN model using PyTorch**

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 100)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

**Instantiate the model, loss function, and optimizer**

In [4]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

**Train the model**

In [5]:
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1} Loss: {loss.item():.4f}')

Epoch 1 Loss: 3.0892
Epoch 2 Loss: 2.9071
Epoch 3 Loss: 2.0878
Epoch 4 Loss: 1.6466
Epoch 5 Loss: 1.5771
Epoch 6 Loss: 1.5669
Epoch 7 Loss: 1.1267
Epoch 8 Loss: 1.0695
Epoch 9 Loss: 0.5365
Epoch 10 Loss: 0.7989
Epoch 11 Loss: 0.3818
Epoch 12 Loss: 0.2113
Epoch 13 Loss: 0.6399
Epoch 14 Loss: 0.3267
Epoch 15 Loss: 1.0353
Epoch 16 Loss: 0.1758
Epoch 17 Loss: 0.2120
Epoch 18 Loss: 0.1862
Epoch 19 Loss: 0.1608
Epoch 20 Loss: 0.6179
Epoch 21 Loss: 0.1827
Epoch 22 Loss: 0.3705
Epoch 23 Loss: 0.0343
Epoch 24 Loss: 0.0593
Epoch 25 Loss: 0.3543
Epoch 26 Loss: 0.0225
Epoch 27 Loss: 0.3521
Epoch 28 Loss: 0.3277
Epoch 29 Loss: 0.0817
Epoch 30 Loss: 0.0693


**Test the model**

In [6]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Testing Accuracy: {100 * correct / total:.2f}%')

Testing Accuracy: 39.09%
